In [123]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Sheet1', header=0 ) 
CSP=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='CSP').iloc[:, 0])
ServiceModelType=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='ServiceModelType' ).iloc[:, 0])
Remediated=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Remediated').iloc[:, 0])
for k in df.columns:
    print(f"d['{k}'] = ''")
lod=[]  
for i in range(1,10):
    d={}
    d['Cloud Service Provider'] = CSP[ random.randint(0, len(CSP)) -1 ]
    d['FedRAMP Package ID'] = f'ID{i}'
    d['Service Type'] = ServiceModelType[ random.randint(0, len(ServiceModelType)) -1 ]
    d['Service Model Type'] = ServiceModelType[ random.randint(0, len(ServiceModelType)) -1 ]
    d['ATO Letter with FedRAMP PMO (Yes or No)'] = 'Yes'
    d['Comments'] = f'Comments{i}'
    lod.append(d)
lod
df=pd.DataFrame(lod) 
write_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx','Sheet1',df)
df=pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Sheet1', header=0 ) 
df


In [ ]:
ip = issue_provider(ctx)  
df=ip.parse_metrics('9311')
df

In [ ]:
ip.driver.quit()

In [ ]:

d={'a':'L', 'b':'M', 'c':'H'}
for i in range(1,11):
    for k in d.keys(): 
        print(f"""
            IF @ANS_7_{i}_{k} > @SUM_111_222_{d[k]}
            BEGIN
                INSERT INTO @ErrorTable  
                SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
                , ('The number provided for ' + identifier_text + ' should not exceed the value of 1.1.1 {d[k]} + 1.1.2 {d[k]} : '  + QuestionText) Error, identifier_text
                FROM vw_OrgSubQuestions WHERE  identifier_text='7.{i}.{k}' AND PK_OrgSubmission = @PK_OrgSubmission  
            END""")

In [ ]:
with open(ctx.get_tempalte_dir()+'sum.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}],SectionNum, SortPos FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO' AND IdText LIKE '4.1._._' ;
""", connstr) 

df['{var}'] = df['{var}'].apply(lambda s : '@SUM_'+re.sub('_[a-z]$','',s))
df['{idt}'] = df['{idt}'].apply(lambda s : f'\'{s}\'') 
df = df.groupby('{var}', as_index=False).agg({'{idt}':list})
df

In [ ]:

df['sql']=[s.replace('{idt}',','.join(x)).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df 

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '2.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 



In [ ]:


df['sql']=s
df['sql']=[x.replace('{idt}',y).replace('{var}',z) for x,y,z in zip(df['sql'], df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df

In [ ]:
df,code=generate_code_from_db(ctx,4019) 
#generate_section(4018, code)
df[:10]
#sql=df_to_sql(df,ctx,template_name='fsma_QuestionsInsert.sql')

In [ ]:
def generate_section(section, code=''): 
    df=sql_todf("SELECT DISTINCT PK_QuestionGroup, ASPX FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO'", connstr).set_index('PK_QuestionGroup' )
    d=df.T.to_dict(orient='records')[0] 
    print(d)
    path=f'C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\{d[section]}'
    with open(path, 'r', encoding='utf-16') as f:
        s=f.read() 
    pname = d[section][d[section].rfind('/')+1:]
    if '<!--questions-->' in s:
        q=re.findall('<!--questions-->.+<!--\/questions-->',s,re.DOTALL)[0]
        s=s.replace(q,f'<!--questions-->\n{code}\n<!--/questions-->')
        with open(f"{ctx.get_dest()}{pname}", 'w', encoding='utf-16') as f:
            f.write(s) 



In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    ip = issue_provider(ctx)
    driver=ip.driver
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
d, lod={'a':'1.1.1', 'b':'1.1.2'},[]
 
for i in range(1,7):
    for k in d.keys(): 
        print(f"""
            IF @ANS_2_1_{i}_{k} > @SUM_{d[k].replace('.','')}
            BEGIN
                INSERT INTO @ErrorTable  
                SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
                , ('The number provided for ' + identifier_text + ' should not exceed the value of {d[k]} : '  + QuestionText) Error, identifier_text
                FROM vw_OrgSubQuestions WHERE  identifier_text='2.1.{i}.{k}' AND PK_OrgSubmission = @PK_OrgSubmission  
            END""")

In [ ]:
df=sql_todf("SELECT DISTINCT PK_QuestionGroup, ASPX FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO'", connstr).set_index('PK_QuestionGroup' )
d=df.T.to_dict(orient='records')[0] 

In [ ]:
re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', '2.1. asdfasdfasdf')[0]

In [ ]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read().replace('\n','')

def met_provider(s):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)  
    return m[0] if len(m) > 0 else ''

def cell_provider(s, i):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}' if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4014", ctx.config['connstr'])
    print(dff.iloc[0])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids()   
list(ids.keys())[3:5] 
 

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 

#df.fillna('', inplace=True)
#df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df.loc[1:,c] = df.loc[1:, 'Question'].apply(lambda s:  cell_provider(s,i)  )

#mets = [str(m)  for m in list(dff['m']) if '2' in str(m) ]  

df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
 
s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('th>','td>',s) 
s=re.sub('<td>Number','<td colspan="2">Number',s)

s=re.sub('<thead>|</thead>|<tbody>|</tbody>','',s) 
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f: 
    s=html.unescape(s) 
    f.write(s)
df[:5]

In [ ]:
dff=df
dff['m']=df['Question'].apply(lambda s:  met_provider(s)  ) 
dff=dff[['Question','m']] 
dff['s']=dff['m'].apply(lambda m: f"'{m}.a.i','{m}.a.ii','{m}.b.i','{m}.b.ii','{m}.c.i','{m}.c.ii'")
lod=[]
pk=36700
sortpos=1
for  i,r in dff[1:].iterrows():
    pk=pk+1
    sortpos=sortpos+1
    m=r['m']
    lod.append({'{idt}':str(m), '{Question}':r['Question'], '{PK}':str(pk), '{sortpos}':str(sortpos), 'FK_QuestionType':'18'})
    idt = f"'{m}.a.i','{m}.a.ii','{m}.b.i','{m}.b.ii','{m}.c.i','{m}.c.ii'".replace('\'','').split(',') 
    for i in idt: 
        sortpos=sortpos+1
        pk=pk+1
        lod.append({'{idt}':str(i), '{Question}':str(i), '{PK}':str(pk), '{sortpos}':str(sortpos), 'FK_QuestionType':'2'})
lod
sql=''
with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
    sql=f.read() 
s=''  
script=''  
for d in lod: 
    s=sql
    for k in d.keys(): 
        s=s.replace(k,d[k])
    script=script+','+s+'\n'
script
with open(f'{ctx.get_dest()}script.sql', 'w') as f:  
    f.write(script)

#print(s)
#dff

In [ ]:
df=sql_todf("SELECT DISTINCT IDTEXT [{idt}], REPLACE(IDTEXT,'.','_') [{var}] FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO' AND PK_QuestionGroup=4014", connstr) 
df[:2]

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    frmval=f.read() 
df['sql'] = [frmval.replace('{idt}',x).replace('{var}',y)  for x,y in zip(df['{idt}'], df['{var}'])]
 
for i,r in df.iterrows():
    if 'ii' in r['{idt}']:
        continue
    d={'a':'H','b':'M','c':'L'}
    for k in d.keys():
        sql=sql+f"""
            IF @SUM_{r['{var}']} > @SUM_111_H
                BEGIN
                    INSERT INTO @ErrorTable  
                    SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
                    , ('The sum provided for ' + identifier_text + ' should not exceed the value of 1.1.1 : '  + QuestionText) Error, identifier_text
                    FROM vw_OrgSubQuestions WHERE  identifier_text='{r['{idt}']}' AND PK_OrgSubmission = @PK_OrgSubmission  
                END
                IF @SUM_{r['{var}']}i > @SUM_112_H
                BEGIN
                    INSERT INTO @ErrorTable  
                    SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
                    , ('The sum provided for ' + identifier_text + ' should not exceed the value of 1.1.2 : '  + QuestionText) Error, identifier_text
                    FROM vw_OrgSubQuestions WHERE  identifier_text='{r['{idt}']}i' AND PK_OrgSubmission = @PK_OrgSubmission  
                END
                
    """
sql
	



In [ ]:


with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql']))) 


In [ ]:

sql=''
lidt=[]
for m in df.iterrows():
    d={'a','H'}
    lidt.append(m)
    lidt.extend(idt.replace('\'','').split(','))
    sql=sql+'\n'+s.replace('{idt}',idt).replace('{var}',m.replace('.','_'))
sql=sql.replace('\'\'','\'')  
lidt  
#print(sql)
#with open(ctx.get_dest()+'script.sql', 'w') as f:
#    f.write( sql ) 

In [ ]:
d, lod={'a':'L', 'b':'M', 'c':'H'},[]
 
for i in range(1,12):
    for k in d.keys(): 
        lod.append({
            'SectionText':'S2'
            ,'MetricXPath':f'//*[contains(@class, \'qid_2_{i}_{k}\')]'
            ,'ErrorAttemptExpression':f'( 1.1.1{d[k]} + 1.1.2{d[k]} ) + 1 '
            ,'ExpectedError':f'2.{i}.{k} should not exceed'
        }) 
df=pd.DataFrame(lod)
df.to_csv(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Selenium\TestData\Q1.csv', index=False)
 

In [ ]:
def qgen(r, ptxt):
    ptxt=re.sub(r'\[\d\]','',ptxt)
    qtext = r[2]  
    qtext=qtext.replace('.a',' FISMA High Systems')
    qtext=qtext.replace('.b',' FISMA Moderate Systems')
    qtext=qtext.replace('.c',' FISMA Low Systems')
    qtext=qtext.replace('.ii',' from 1.1.2')
    qtext=qtext.replace('.i',' from 1.1.1') 
    # ptxt = ''.join(s for s in ptxt.split(' ')[1:]) 
    # ptxt + ' ' +
    return qtext
qgen( ('1','2','2.10.2.a.ii')   , '2.10.2 How many systems identified in question 2.10 provide an option for MFA credentials susceptible to phishing[2]' )

In [ ]:


df=sql_todf("SELECT DISTINCT PK_Question [{PK}], IDTEXT [{idt}], REPLACE(QText,'.','.') [{QuestionText}], sortpos [{sortpos}], FK_QuestionType [{FK_QuestionType}] FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO' AND PK_QuestionGroup=4014", connstr) 
sql=''
with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
    sql=f.read()  
r, l, pq, pqt=[], [],'','' 
for i,r in df[:].iterrows():
    if len(r['{QuestionText}']) > 20 and pqt != r['{QuestionText}']:
        pqt = r['{QuestionText}']
        #print(pqt) 
    s=sql
    s=s.replace('{QuestionText}',qgen(r, pqt) )
    for c in df.columns:
        s=s.replace(c, str(r[c])) 
        
    l.append(s)  
s='\n,'.join(l)
print(s) 
